# Function Analysis (JSON): Report number of slices used per mouse

In [1]:
import os
import csv
import json
import fnmatch
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_jsons(dirname, expt):
    json_paths = []
    for jfile in os.listdir(dirname):
        if fnmatch.fnmatch(jfile, '*.%s.json' %expt):
            jpath = os.path.join(dirname, jfile)
            json_paths.append(jpath)
    return json_paths

In [3]:
#make list of json paths for every json file in MIES Experiments directory on 279
json_list = get_jsons("//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/", "PS")
json_list

['//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Sim1-Cre_KJ18;Ai139;Sst-IRES-FlpO;Ai65F-386176.11.06.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Rbp4-Cre_KL100;Ai14-357010.03.01.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Oxtr-T2A-Cre;Pvalb-T2A-FlpO;Ai65-370683.03.01.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Ntsr1-Cre_GN220;Ai14-361815.05.01.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Gad2-IRES-Cre;Ai14-296621.05.02.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Cux2-CreERT2;Ai14-315357.03.02.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/Rbp4-Cre_KL100;Ai14-280355.05.01.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all-metadata-files/H17.06.015.13.08.PS.json',
 '//allen/programs/celltypes/workgroups/279/Patch-Seq/all

In [4]:
from pandas.io.json import json_normalize

def flatten_attempts(slice_info, form_version):
    """Return flattened slice metadata dataframe.
    
    Parameters
    ----------
    slice_info : dict
        A dictionary of slices with nested pipette attempts.
    
    form_version : string
        A string containing the JEM form version.
        (Pre-version 2 contains IVSCC, PatchSeq and Electroporation arrays)
    Returns
    -------
    attempts_slice_df : pandas dataframe
        A dataframe of all pipette attempts along with all slice metadata.
    """
    
    df = json_normalize(slice_info)
    if form_version >= "2":
        ps_array_name = "pipettes"
    else:
        ps_array_name = "pipettesPatchSeqPilot"
    try:
        attempts_df = json_normalize(slice_info[ps_array_name])
        attempts_df["limsSpecName"] = df["limsSpecName"].values[0]
        attempts_df["attempt"] = [p+1 for p in attempts_df.index.values]
        attempts_slice_df = pd.merge(df, attempts_df, how="outer", on="limsSpecName")
        attempts_slice_df.drop(ps_array_name, axis=1, inplace=True)
        return attempts_slice_df
    except IndexError:
        pass

def is_field(df, colname):
    """Determine whether a column name exists in a dataframe.
    
    Parameters
    ----------
    df : a Pandas dataframe
    colname : string
        
    Returns
    -------
    Boolean
        Boolean value indicating if the colname exists in the dataframe.
    """
    
    try:
        df[colname]
        return True
    except KeyError:
        return False

### Output Starts

In [5]:
json_df = pd.DataFrame()

for json_path in json_list:
    with open(json_path) as data_file:
        slice_info = json.load(data_file)
        if is_field(slice_info, "formVersion"):
            jem_version = slice_info["formVersion"]
        else:
            jem_version = "1.0.0"
        flat_df = flatten_attempts(slice_info, jem_version)
        json_df = pd.concat([json_df, flat_df], axis=0)
        
pd.options.display.max_columns = None  #Displays all the columns 
json_df.head()

acsfProductionDate approach.anatomicalLocation approach.autoRoi  \
0         2018-04-18                         NaN              NaN   
1         2018-04-18                         NaN              NaN   
2         2018-04-18                         NaN              NaN   
3         2018-04-18                         NaN              NaN   
4         2018-04-18                         NaN              NaN   

  approach.cellHealth approach.corticalLayer approach.creCell  approach.depth  \
0                   3                    NaN             Cre+             NaN   
1                   2                    NaN             Cre+             NaN   
2                   2                    NaN             Cre+             NaN   
3                   2                    NaN             Cre+             NaN   
4                   2                    NaN             Cre+             NaN   

  approach.detailedLocation approach.manualRoi approach.otherPilotName  \
0                       NaN                NaN                     NaN   
1                       NaN                NaN                     NaN   
2                       NaN                NaN                     NaN   
3                       NaN                NaN                     NaN   
4                       NaN                NaN                     NaN   

  approach.pilotName approach.pilotTest01 approach.pilotTest03  \
0               None                  NaN                  NaN   
1               None                  NaN                  NaN   
2               None                  NaN                  NaN   
3               None                  NaN                  NaN   
4               None                  NaN                  NaN   

  approach.pilotTest04 approach.pilotTest05 approach.sliceHealth  attempt  \
0                  NaN                  NaN                    2      1.0   
1                  NaN                  NaN                    2      2.0   
2                  NaN                  NaN                    2      3.0   
3                  NaN                  NaN                    2      4.0   
4                  NaN                  NaN                    2      5.0   

  autoRoi badSweeps blankFillDate                        date depth  \
0    None       NaN    2018-03-22  2018-04-18 13:17:02 -07:00   -50   
1    None       NaN    2018-03-22  2018-04-18 13:17:02 -07:00   -50   
2    None       NaN    2018-03-22  2018-04-18 13:17:02 -07:00   -50   
3    None       NaN    2018-03-22  2018-04-18 13:17:02 -07:00   -50   
4    None       NaN    2018-03-22  2018-04-18 13:17:02 -07:00   -50   

  experimentType extraction.bfiWitness extraction.endPipetteR  \
0            NaN                   NaN                   1000   
1            NaN                   NaN                   1000   
2            NaN                   NaN                   1000   
3            NaN                   NaN                   1000   
4            NaN                   NaN                   1000   

  extraction.extractionNotes extraction.extractionObservations  \
0                        NaN                               NaN   
1                        NaN                               NaN   
2                        NaN                               NaN   
3                        NaN                               NaN   
4                        NaN                               NaN   

  extraction.nucleus extraction.postPatch  extraction.pressureApplied  \
0                 no       nucleus_absent                       -80.0   
1                NaN      nucleus_present                       -80.0   
2                 no       nucleus_absent                       -80.0   
3  not_intentionally      nucleus_present                       -80.0   
4                NaN       nucleus_absent                       -80.0   

   extraction.retractionPressureApplied extraction.sampleObservations  \
0                                 -80.0                           Na

### Trying to only have date as YYYY-MM-DD

In [6]:
json_df['date'] = json_df['date'].str[:11]

In [7]:
json_df.tail()

acsfProductionDate approach.anatomicalLocation approach.autoRoi  \
1           12/15/16                          V1              NaN   
2           12/15/16                          V1              NaN   
3           12/15/16                          V1              NaN   
0           08/08/17                          V1              NaN   
1           08/08/17                          V1              NaN   

  approach.cellHealth approach.corticalLayer approach.creCell  approach.depth  \
1                 NaN                    NaN             Cre+            53.1   
2                 NaN                    NaN             Cre+            49.1   
3                 NaN                    NaN             Cre+             NaN   
0                 NaN                    2/3             Cre+           118.8   
1                 NaN                    2/3             Cre+           105.5   

  approach.detailedLocation approach.manualRoi approach.otherPilotName  \
1              more lateral                NaN                     NaN   
2              more lateral                NaN                     NaN   
3              more lateral                NaN                     NaN   
0              most lateral                NaN                     NaN   
1               more medial                NaN                     NaN   

  approach.pilotName approach.pilotTest01 approach.pilotTest03  \
1     3 Minute Pilot                  NaN                  NaN   
2     3 Minute Pilot                  NaN                  NaN   
3     3 Minute Pilot                  NaN                  NaN   
0                NaN                  NaN                  NaN   
1                NaN                  NaN                  NaN   

  approach.pilotTest04 approach.pilotTest05 approach.sliceHealth  attempt  \
1                  NaN                  NaN                    4      2.0   
2                  NaN                  NaN                    4      3.0   
3                  NaN                  NaN                    4      4.0   
0                  NaN                  NaN                    3      1.0   
1                  NaN                  NaN                    2      2.0   

  autoRoi badSweeps blankFillDate         date depth   experimentType  \
1     NaN       NaN           NaN  12/16/2016    NaN  Patch-Seq Pilot   
2     NaN       NaN           NaN  12/16/2016    NaN  Patch-Seq Pilot   
3     NaN       NaN           NaN  12/16/2016    NaN  Patch-Seq Pilot   
0     NaN       NaN           NaN  08/09/2017    NaN  Patch-Seq Pilot   
1     NaN       NaN           NaN  08/09/2017    NaN  Patch-Seq Pilot   

  extraction.bfiWitness extraction.endPipetteR  \
1                   NaN                    NaN   
2                   NaN                    NaN   
3                   NaN                    NaN   
0                   NaN                    NaN   
1                   NaN                    NaN   

            extraction.extractionNotes    extraction.extractionObservations  \
1                                  NaN  Fluorescence in Pipette,Cell Dimmed   
2                                  NaN  Fluorescence in Pipette,Cell Dimmed   
3                                  NaN  Fluorescence in Pipette,Cell Dimmed   
0  0.15 units of pressure on expulsion                             Too Deep   
1  0.15 units of pressure on expulsion                             Too Deep   

  extraction.nucleus extraction.postPatch  extraction.pressureApplied  \
1                NaN          Outside-Out                        80.0   
2                NaN          Outside-Out                        80.0   
3                NaN          Outside-Out                        80.0   
0                NaN              No-Seal                       -25.0   
1                NaN              No-Seal                       -30.0   

   extraction.retractionPressureApplied extraction.sampleObservations  \
1                                   NaN                           Na

### String Slicing last .##.## in limsSpecName to correct counts

In [8]:
#json_df['limsSpecName'] = json_df['limsSpecName'].str[:-3]

In [9]:
json_df.tail()

acsfProductionDate approach.anatomicalLocation approach.autoRoi  \
1           12/15/16                          V1              NaN   
2           12/15/16                          V1              NaN   
3           12/15/16                          V1              NaN   
0           08/08/17                          V1              NaN   
1           08/08/17                          V1              NaN   

  approach.cellHealth approach.corticalLayer approach.creCell  approach.depth  \
1                 NaN                    NaN             Cre+            53.1   
2                 NaN                    NaN             Cre+            49.1   
3                 NaN                    NaN             Cre+             NaN   
0                 NaN                    2/3             Cre+           118.8   
1                 NaN                    2/3             Cre+           105.5   

  approach.detailedLocation approach.manualRoi approach.otherPilotName  \
1              more lateral                NaN                     NaN   
2              more lateral                NaN                     NaN   
3              more lateral                NaN                     NaN   
0              most lateral                NaN                     NaN   
1               more medial                NaN                     NaN   

  approach.pilotName approach.pilotTest01 approach.pilotTest03  \
1     3 Minute Pilot                  NaN                  NaN   
2     3 Minute Pilot                  NaN                  NaN   
3     3 Minute Pilot                  NaN                  NaN   
0                NaN                  NaN                  NaN   
1                NaN                  NaN                  NaN   

  approach.pilotTest04 approach.pilotTest05 approach.sliceHealth  attempt  \
1                  NaN                  NaN                    4      2.0   
2                  NaN                  NaN                    4      3.0   
3                  NaN                  NaN                    4      4.0   
0                  NaN                  NaN                    3      1.0   
1                  NaN                  NaN                    2      2.0   

  autoRoi badSweeps blankFillDate         date depth   experimentType  \
1     NaN       NaN           NaN  12/16/2016    NaN  Patch-Seq Pilot   
2     NaN       NaN           NaN  12/16/2016    NaN  Patch-Seq Pilot   
3     NaN       NaN           NaN  12/16/2016    NaN  Patch-Seq Pilot   
0     NaN       NaN           NaN  08/09/2017    NaN  Patch-Seq Pilot   
1     NaN       NaN           NaN  08/09/2017    NaN  Patch-Seq Pilot   

  extraction.bfiWitness extraction.endPipetteR  \
1                   NaN                    NaN   
2                   NaN                    NaN   
3                   NaN                    NaN   
0                   NaN                    NaN   
1                   NaN                    NaN   

            extraction.extractionNotes    extraction.extractionObservations  \
1                                  NaN  Fluorescence in Pipette,Cell Dimmed   
2                                  NaN  Fluorescence in Pipette,Cell Dimmed   
3                                  NaN  Fluorescence in Pipette,Cell Dimmed   
0  0.15 units of pressure on expulsion                             Too Deep   
1  0.15 units of pressure on expulsion                             Too Deep   

  extraction.nucleus extraction.postPatch  extraction.pressureApplied  \
1                NaN          Outside-Out                        80.0   
2                NaN          Outside-Out                        80.0   
3                NaN          Outside-Out                        80.0   
0                NaN              No-Seal                       -25.0   
1                NaN              No-Seal                       -30.0   

   extraction.retractionPressureApplied extraction.sampleObservations  \
1                                   NaN                           Na

### Sets the Date column as the index

In [10]:
json_df.set_index('date', inplace = True)

### Sorts the Date column

In [11]:
json_df.sort_index(inplace = True)

### Creating a count column from counting limsSpecname

In [13]:
json_df['Count'] = json_df.groupby('limsSpecName')['limsSpecName'].transform('count')
#json_df['Count'] = json_df.groupby('limsSpecName')['limsSpecName'].transform('count')
#json_df['Count'] = json_df.groupby('limsSpecName').nunique()
#df['count'] = df.groupby('group')['group'].transform('count')

In [14]:
json_df.tail()

acsfProductionDate approach.anatomicalLocation approach.autoRoi  \
date                                                                          
2018-05-09          2018-05-08                         NaN              NaN   
2018-05-09          2018-05-08                         NaN              NaN   
2018-05-09          2018-05-08                         NaN              NaN   
2018-05-09          2018-05-08                         NaN              NaN   
2018-05-09          2018-05-08                         NaN              NaN   

            approach.cellHealth approach.corticalLayer approach.creCell  \
date                                                                      
2018-05-09                    3                    NaN             Cre+   
2018-05-09                    2                    NaN             Cre+   
2018-05-09                    1                    NaN             Cre+   
2018-05-09                    2                    NaN             Cre+   
2018-05-09                    3                    NaN             Cre+   

             approach.depth approach.detailedLocation approach.manualRoi  \
date                                                                       
2018-05-09              NaN                       NaN                NaN   
2018-05-09              NaN                       NaN                NaN   
2018-05-09              NaN                       NaN                NaN   
2018-05-09              NaN                       NaN                NaN   
2018-05-09              NaN                       NaN                NaN   

            approach.otherPilotName approach.pilotName approach.pilotTest01  \
date                                                                          
2018-05-09                      NaN               None                  NaN   
2018-05-09                      NaN               None                  NaN   
2018-05-09                      NaN               None                  NaN   
2018-05-09                      NaN               None                  NaN   
2018-05-09                      NaN               None                  NaN   

            approach.pilotTest03 approach.pilotTest04 approach.pilotTest05  \
date                                                                         
2018-05-09                   NaN                  NaN                  NaN   
2018-05-09                   NaN                  NaN                  NaN   
2018-05-09                   NaN                  NaN                  NaN   
2018-05-09                   NaN                  NaN                  NaN   
2018-05-09                   NaN                  NaN                  NaN   

            approach.sliceHealth  attempt autoRoi badSweeps blankFillDate  \
date                                                                        
2018-05-09                     2      1.0    None       NaN    2018-03-22   
2018-05-09                     2      2.0    None       NaN    2018-03-22   
2018-05-09                     3      1.0    None       NaN    2018-03-22   
2018-05-09                     2      1.0    None       NaN    2018-04-29   
2018-05-09                     2      3.0    None       NaN    2018-03-22   

            depth experimentType extraction.bfiWitness extraction.endPipetteR  \
date                                                                            
2018-05-09     52            NaN                   NaN                   2000   
2018-05-09     40            NaN                   NaN                   2000   
2018-05-09     58            NaN                   NaN                    NaN   
2018-05-09      0            NaN                   NaN                    NaN   
2018-05-09     81            NaN                   NaN                      6   

            extraction.extractionNotes extraction.extractionObservations  \
date                                                                       
2018-05-09                          1%

In [15]:
json_df1 = json_df['2018-05-07' : '2018-05-08']
#['Start date' : 'End date']

In [16]:
#json_df1

### Pivot Tables

In [23]:
json_df2 = json_df1.pivot_table(index = 'limsSpecName', columns = 'date', values = 'Count') 

In [24]:
json_df2

date                                                2018-05-07 
limsSpecName                                                   
Rbp4-Cre_KL100;Ai14-388821.03.01                              1
Rbp4-Cre_KL100;Ai14-388821.03.02                              2
Rbp4-Cre_KL100;Ai14-388821.04.01                              3
Rbp4-Cre_KL100;Ai14-388821.04.02                              1
Vip-IRES-Cre;Ai14-388042.02.01                                2
Vip-IRES-Cre;Ai14-388042.03.01                                5
Vip-IRES-Cre;Ai14-388042.03.02                                3
Vip-IRES-Cre;Ai14-388042.04.02                                4
Vip-IRES-Cre;Ai14-390084.03.01                                4
Vip-IRES-Cre;Ai14-390084.04.02                                3
Vipr2-IRES2-Cre;Slc32a1-T2A-FlpO;Ai65-388176.02.01            2
Vipr2-IRES2-Cre;Slc32a1-T2A-FlpO;Ai65-388176.03.01            1
Vipr2-IRES2-Cre;Slc32a1-T2A-FlpO;Ai65-388176.03.02            2
Vipr2-IRES2-Cre;Slc32a1-T2A-FlpO;Ai65-388176.04.02            1

In [25]:
json_df3 = json_df1.pivot_table(index = 'limsSpecName', columns = ['date', 'rigOperator'], values = 'Count')

In [26]:
json_df3

date                                               2018-05-07              \
rigOperator                                           lindsayn lisak ramr   
limsSpecName                                                                
Rbp4-Cre_KL100;Ai14-388821.03.01                           NaN   NaN  1.0   
Rbp4-Cre_KL100;Ai14-388821.03.02                           2.0   NaN  NaN   
Rbp4-Cre_KL100;Ai14-388821.04.01                           NaN   NaN  NaN   
Rbp4-Cre_KL100;Ai14-388821.04.02                           NaN   NaN  1.0   
Vip-IRES-Cre;Ai14-388042.02.01                             NaN   NaN  2.0   
Vip-IRES-Cre;Ai14-388042.03.01                             NaN   5.0  NaN   
Vip-IRES-Cre;Ai14-388042.03.02                             NaN   NaN  NaN   
Vip-IRES-Cre;Ai14-388042.04.02                             4.0   NaN  NaN   
Vip-IRES-Cre;Ai14-390084.03.01                             NaN   4.0  NaN   
Vip-IRES-Cre;Ai14-390084.04.02                             3.0   NaN  NaN   
Vipr2-IRES2-Cre;Slc32a1-T2A-FlpO;Ai65-388176.02.01         NaN   2.0  NaN   
Vipr2-IRES2-Cre;Slc32a1-T2A-FlpO;Ai65-388176.03.01         NaN   NaN  1.0   
Vipr2-IRES2-Cre;Slc32a1-T2A-FlpO;Ai65-388176.03.02         2.0   NaN  NaN   
Vipr2-IRES2-Cre;Slc32a1-T2A-FlpO;Ai65-388176.04.02         NaN   NaN  1.0   

date                                                       
rigOperator                                        rustym  
limsSpecName                                               
Rbp4-Cre_KL100;Ai14-388821.03.01                      NaN  
Rbp4-Cre_KL100;Ai14-388821.03.02                      NaN  
Rbp4-Cre_KL100;Ai14-388821.04.01                      3.0  
Rbp4-Cre_KL100;Ai14-388821.04.02                      NaN  
Vip-IRES-Cre;Ai14-388042.02.01                        NaN  
Vip-IRES-Cre;Ai14-388042.03.01                        NaN  
Vip-IRES-Cre;Ai14-388042.03.02                        3.0  
Vip-IRES-Cre;Ai14-388042.04.02                        NaN  
Vip-IRES-Cre;Ai14-390084.03.01                        NaN  
Vip-IRES-Cre;Ai14-390084.04.02                        NaN  
Vipr2-IRES2-Cre;Slc32a1-T2A-FlpO;Ai65-388176.02.01    NaN  
Vipr2-IRES2-Cre;Slc32a1-T2A-FlpO;Ai65-388176.03.01    NaN  
Vipr2-IRES2-Cre;Slc32a1-T2A-FlpO;Ai65-388176.03.02    NaN  
Vipr2-IRES2-Cre;Slc32a1-T2A-FlpO;Ai65-388176.04.02    NaN

In [27]:
"""
Problem: Count uses limsSpecName which repeats on every attempt so
the count only displays number of attempts. Even if you slice the last 
digits of the limsSpecName, the count is unaffected. 

Solution: Creating a for loop to count repeated values as one.

Somehow using Histology well ID because a slice can only have one.


"""

'\nProblem: Count uses limsSpecName which repeats on every attempt so\nthe count only displays number of attempts. Even if you slice the last \ndigits of the limsSpecName, the count is unaffected. \n\nSolution: Creating a for loop to count repeated values as one.\n\nSomehow using Histology well ID because a slice can only have one.\n\n\n'